In [10]:
#Imports
import json
import joblib
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge,  LogisticRegression
from sklearn.metrics import (
                            mean_absolute_error, mean_squared_error, r2_score,
                            accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
                            )
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print("Import loaded")

Import loaded


#### 01) Load Prerocessed Arrays

In [11]:
ARTIFACTS_DIR = Path("artifacts")
FIGURES_DIR = Path("artifacts/figures")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

#### 02) Load Preprocessed Arrays

In [12]:
X_train_reg = np.load(ARTIFACTS_DIR / 'X_train_reg.npz')['data']
X_test_reg = np.load(ARTIFACTS_DIR / 'X_test_reg.npz')['data']
y_train_reg = np.load(ARTIFACTS_DIR / 'y_train_reg.npz')['data']
y_test_reg = np.load(ARTIFACTS_DIR / 'y_test_reg.npz')['data']

X_train_cls = np.load(ARTIFACTS_DIR / 'X_train_cls.npz')['data']
X_test_cls = np.load(ARTIFACTS_DIR / 'X_test_cls.npz')['data']
y_train_cls = np.load(ARTIFACTS_DIR / 'y_train_cls.npz')['data']
y_test_cls = np.load(ARTIFACTS_DIR / 'y_test_cls.npz')['data']


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\X_train_reg.npz'